In [1]:
from utils.molecule_tools import get_molecule
from utils.cs_vqe_tools_original import greedy_dfs
import json

/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/__init__.py:86: DeprecationWarning: The package qiskit.aqua is deprecated. It was moved/refactored to qiskit-terra For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  warn_package('aqua', 'qiskit-terra')
/home/tim/anaconda3/lib/python3.7/site-packages/qiskit/aqua/components/optimizers/optimizer.py:50: DeprecationWarning: The package qiskit.aqua.components.optimizers is deprecated. It was moved/refactored to qiskit.algorithms.optimizers (pip install qiskit-terra). For more information see <https://github.com/Qiskit/qiskit-aqua/blob/main/README.md#migration-guide>
  'qiskit.algorithms.optimizers', 'qiskit-terra')


In [16]:
file = 'molecule_data'
with open('data/'+file+'.json', 'r') as json_file:
    molecule_data = json.load(json_file)

speciesname_list = list(molecule_data.keys())
len(speciesname_list)

['O_STO-3G_SINGLET',
 'CH2_STO-3G_SINGLET',
 'BeH2_STO-3G_SINGLET',
 'Be_STO-3G_SINGLET',
 'C_STO-3G_SINGLET']

In [24]:
#model_data = {}
for speciesname in ['O_STO-3G_SINGLET']:
    print(speciesname)
    molecule = get_molecule(speciesname, taper=True)
    num_qubits  = molecule['num_qubits']
    ham         = molecule['hamiltonian']
    uccsd       = molecule['uccsdansatz']
    hf_config   = molecule['hf_config']
    num_tapered = molecule['num_tapered']
    true_gs_nrg = molecule['true_gs_nrg']
    true_gs_vec = molecule['true_gs_vec']
    
    for p in ham.keys():
        ham[p] = ham[p].real    
    for p in uccsd.keys():
        uccsd[p] = uccsd[p].imag
    
    terms_noncon = greedy_dfs(ham, 10, criterion='size')[-1]
    
    model_data[speciesname] = {"ham":ham,
                               "uccsd":uccsd,
                               "num_qubits":num_qubits,
                               "hf_config":hf_config,
                               "terms_noncon":terms_noncon,
                               "num_tapered":num_tapered,
                               "true_gs_nrg":true_gs_nrg}
                               #"true_gs_vec":true_gs_vec}

O_STO-3G_SINGLET
*** sector saved, will check tapered ground state energy matches target problem ***
Attempting to taper 10 --> 5 qubits
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Energies match in sector (1, -1, -1, 1, 1), tapering successful!



TypeError: 'PauliOp' object is not iterable

In [26]:
len(model_data)

23

In [27]:
data={}
for k in model_data.keys():
    data[str(k)] = model_data[k]
     
with open("data/model_data.json", "w") as outfile: 
    json.dump(data, outfile)

In [21]:
len(speciesname_list)

24